In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date


loading in data 


In [5]:
os.getcwd()
os.chdir("..\RA_police\Injuries")

'c:\\Users\\Olivia_Pinney\\Documents\\UChicago_Materials\\RA_police\\Injuries'

In [12]:
data_path_iigh={
  "history_profiles":"chicago-police-data/data/unified_data/profiles/final-profiles.csv",
  "history_roster":"chicago-police-data/data/unified_data/roster/roster_1936-2017_2017-04.csv",
  "history_unit":"chicago-police-data/data/unified_data/unit-history/unit-history.csv", 
  "trr_overall":"chicago-police-data/data/unified_data/TRR/TRR-main_2004-2016_2016-09.csv",
  "trr_subjects":"chicago-police-data/data/unified_data/TRR/TRR-subjects_2004-2016_2016-09.csv",
  "trr_officers":"chicago-police-data/data/unified_data/TRR/TRR-officers_2004-2016_2016-09.csv",
  "trr_actions":"chicago-police-data/data/unified_data/TRR/TRR-actions-responses_2004-2016_2016-09.csv",
  "complaints":"chicago-police-data/data/unified_data/complaints/complaints-accused.csv", 
  "complaints_detailed":"chicago-police-data/data/unified_data/complaints/complaints-complaints.csv",
  "complaint_types":"chicago-police-data/data/context_data/Complaint Categories.csv", 
  #the following are internal file paths
  "dat":"internal_data/stacked_officer_arrests_01to18.csv",
  "dat2":"internal_data/injuries_3_2022.csv",
  "dat3":"internal_data/injuries_10_2022.csv"
}

# Section on Histories

In [41]:
#merge history
history_profiles=pd.read_csv(data_path_iigh["history_profiles"])
history_roster=pd.read_csv(data_path_iigh["history_roster"])
history_unit=pd.read_csv(data_path_iigh["history_unit"]) 

#print(history_profiles.columns) #UID and personal information
#print(history_roster.columns) #UID - smaller dataset
#print(history_unit.columns) #UID, and their time on each unit

C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_21952\2184588313.py:2: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  history_profiles=pd.read_csv(data_path_iigh["history_profiles"])


In [134]:
df=history_roster #[history_roster["current_status"]==1]
#df["resignation_year"]
df["resignation_date"]=df["resignation_date"].fillna("2999-01-01")
df["resignation_year"]=df["resignation_date"].apply(lambda x: int(str(x)[0:4]))

df["appointed_date"]=df["appointed_date"].fillna("1000-01-01")
df["appointed_year"]=df["appointed_date"].apply(lambda x: int(str(x)[0:4]))

df=df[df["appointed_year"]<2015] #officer in 2014
df=df[df["resignation_year"]>2013] #officer in 2014
df=df[df["current_status"]==1] #latest position

df.columns


Index(['row_id', 'gender', 'race', 'birth_year', 'current_age',
       'current_status', 'appointed_date', 'rank_no', 'current_rank',
       'current_unit', 'unit_description', 'resignation_date', 'star1',
       'star2', 'star3', 'star4', 'star5', 'star6', 'star7', 'star8', 'star9',
       'star10', 'first_name', 'first_name_NS', 'last_name', 'last_name_NS',
       'middle_initial', 'middle_initial2', 'suffix_name', 'merge',
       'roster_1936-2017_2017-04_ID', 'UID', 'old_UID', 'link_UID',
       'resignation_year', 'appointed_year'],
      dtype='object')

In [144]:
temp=df[["unit_description"]].value_counts(sort=True)
temp=temp.reset_index()

temp.to_csv("departments.csv",index=False)

# Section of Complaints

In [135]:
#merge complaints
complaints=pd.read_csv(data_path_iigh["complaints"])
complaints_detailed=pd.read_csv(data_path_iigh["complaints_detailed"])
complaint_types=pd.read_csv(data_path_iigh["complaint_types"])

cmp= complaints.merge(complaints_detailed[["cr_id", "complaint_date"]],on="cr_id",how="left")
cmp["code"]=cmp["complaint_code"]
complaint_types=complaint_types[~pd.isna(complaint_types["111"])]
complaint_types["code"]=complaint_types["111"]
cmp= cmp.merge(complaint_types,on="code",how="left")

C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_21952\3238190781.py:2: DtypeWarning: Columns (3,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints=pd.read_csv(data_path_iigh["complaints"])
C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_21952\3238190781.py:3: DtypeWarning: Columns (0,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_detailed=pd.read_csv(data_path_iigh["complaints_detailed"])


In [136]:
#cmp["UID","CITIZEN / DEPT",].value_counts()

cmp=cmp[cmp["UID"].isin(df["UID"])]
cmp=cmp[cmp["CITIZEN / DEPT"].isin(["CITIZEN","DEPT"])]



In [137]:
#eventually want this
#cmp["complaint_category"].value_counts()

temp=cmp[["UID","CITIZEN / DEPT"]].value_counts().reset_index()
temp = temp.set_axis(['UID', 'CITIZEN / DEPT', 'COUNT'], axis=1, inplace=False)

cmp_final=temp.pivot(index="UID",columns="CITIZEN / DEPT",values="COUNT").reset_index()
cmp_final.fillna(0.0)
df=df.merge(cmp_final,on="UID")

CITIZEN / DEPT,UID,CITIZEN,DEPT
0,100007.0,8.0,0.0
1,100009.0,1.0,1.0
2,100011.0,16.0,2.0
3,100012.0,3.0,2.0
4,100013.0,12.0,11.0
...,...,...,...
9535,131785.0,9.0,5.0
9536,131786.0,6.0,4.0
9537,131787.0,1.0,3.0
9538,131788.0,2.0,1.0


## TTR

In [119]:
#merge trr
trr_overall=pd.read_csv(data_path_iigh["trr_overall"])
trr_subjects=pd.read_csv(data_path_iigh["trr_subjects"])
trr_officers=pd.read_csv(data_path_iigh["trr_officers"])
trr_actions=pd.read_csv(data_path_iigh["trr_actions"])

#trr=trr_overall.merge(trr_actions,on="trr_id") #,how="left"
trr=trr_overall.merge(trr_officers,on="trr_id") #,how="left"
trr=trr.merge(trr_subjects,on="trr_id") #,how="left"


In [124]:

trr["trr_year"]=trr["trr_date_x"].apply(lambda x: int(str(x)[0:4]))
trr_14=trr[trr["trr_year"]==2014]


In [120]:
#come back to this

trr_actions[trr_actions["person"] == "Subject Action"]["action"].value_counts(sort=True,dropna=False)
#trr_actions[trr_actions["person"] == "Subject Action"]["resistance_type"].value_counts(sort=True,dropna=False)

#"Passive Resister"
#"Active Resister"



DID NOT FOLLOW VERBAL DIRECTION                           61833
PULLED AWAY                                               52458
OTHER (SPECIFY)                                           36366
STIFFENED (DEAD WEIGHT)                                   35248
IMMINENT THREAT OF BATTERY                                24033
FLED                                                      21440
ATTACK WITHOUT WEAPON                                     15545
ATTACK WITH WEAPON                                         1716
USES FORCE LIKELY TO CAUSE DEATH OR GREAT BODILY HARM      1605
WEAPON(DESCRIBE IN ADDITIONAL INFO)                        1243
Name: action, dtype: int64

In [389]:
df["altercation"]=False
df["altercation"][df["UID"].isin(trr_14["UID"])]=True

df["civ_complaint"]=False
df["dept_complaint"]=False

df["civ_complaint"][df["CITIZEN"]>0]=True
df["dept_complaint"][df["DEPT"]>0]=True

df["dept_complaint"].value_counts()


-2690

In [ ]:
temp=df[["unit_description","altercation","civ_complaint","dept_complaint"]].value_counts(sort=True)
temp=temp.reset_index()

temp.to_csv("departments_granular.csv",index=False)




